In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 10 01:48:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
!pip install transformers
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.0 MB/s 
     |████████████████████████████████| 7.6 MB 70.9 MB/s 
     |████████████████████████████████| 163 kB 80.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 1.3 MB 34.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=279a83bde2b7132dcd820d26439b8f4a2aafe00cbd71cb44ac86cb3ef3db7e44
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [5]:
import numpy as np
import pandas as pd

In [9]:
df_reviews=pd.read_csv('/content/drive/MyDrive/6402/2018_yelp_review_matched.csv', usecols=['business_id', 'text'], nrows=10000)


In [10]:
df_reviews.shape

(10000, 2)

In [8]:
from sentence_transformers import SentenceTransformer, LoggingHandler
import logging
import pickle

In [12]:
#Important, you need to shield your code with if __name__. Otherwise, CUDA runs into issues when spawning new processes.
if __name__ == '__main__':

    #Define the model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    #Start the multi-process pool on all available CUDA devices
    pool = model.start_multi_process_pool()

    #Compute the embeddings using the multi-process pool
    emb = model.encode_multi_process(df_reviews['text'].tolist(), pool)

    print("Embeddings computed. Shape:", emb.shape) 
    df_out_raw = pd.concat([df_reviews, pd.DataFrame(emb, columns=['sbert_'+str(i) for i in range(len(emb[0]))])], axis = 1)
    df_out = df_out_raw.groupby('business_id').mean()
    Optional: Stop the proccesses in the pool
    model.stop_multi_process_pool(pool)

    
    with open('/content/drive/MyDrive/6402/embed_all', 'wb') as f: 
        pickle.dump(df_out_raw, f) 


In [13]:
df_out_raw.tail()

,business_id,text,sbert_0,sbert_1,sbert_2,sbert_3,sbert_4,sbert_5,sbert_6,sbert_7,...,sbert_374,sbert_375,sbert_376,sbert_377,sbert_378,sbert_379,sbert_380,sbert_381,sbert_382,sbert_383
9995,QS3QxI7u5PRdtbGgI0-UsA,Great happy hour menu. The rolls are pretty st...,-0.090557,0.024446,-0.004708,0.047466,-0.172598,-0.016801,-0.035726,-0.052257,...,0.021409,0.007226,0.060769,-0.022210,0.087626,0.090392,0.032394,0.001370,-0.036370,0.014739
9996,QS3QxI7u5PRdtbGgI0-UsA,Lunchtime Sushi!!! And make sure to get there...,-0.027748,0.059063,0.068863,0.071952,-0.045562,-0.026167,0.005291,-0.042645,...,0.011303,0.078489,0.010702,0.028093,0.041628,0.094229,-0.029066,-0.017899,-0.093262,0.057501
9997,QS3QxI7u5PRdtbGgI0-UsA,This is one of the best sushi spots in the Val...,-0.053731,0.030905,0.002242,0.077206,-0.082543,0.013734,0.035699,-0.084823,...,0.016079,0.087569,0.036808,-0.058746,0.066512,0.152301,0.033198,0.008768,-0.056714,0.066556
9998,QS3QxI7u5PRdtbGgI0-UsA,We were surprised at how good the sushi was he...,-0.060910,0.077863,0.068812,0.055402,-0.058351,0.020273,-0.002198,-0.025635,...,0.000453,0.068363,0.045547,-0.042756,-0.005258,0.054223,0.052572,-0.010344,-0.079159,0.079102
9999,QS3QxI7u5PRdtbGgI0-UsA,Always fresh and great sushi ! Fast and friend...,-0.101765,-0.006277,0.024221,0.067548,-0.054033,0.022168,-0.021775,-0.087450,...,0.035905,0.079298,0.065176,0.012923,0.064117,0.114940,0.075502,0.023469,-0.057464,0.032893
